In [1]:
from redcap import Project
import pandas as pd

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = '63A451E2DA73C4D6A4C64A17B36B9565'
project = Project(api_url, api_key)

In [2]:
##Extract all data from Maternal Supplementation.
MS_data = project.export_records(forms=['maternal_supplementation'])

#convert data to Dataframe.
MatSup_data=pd.DataFrame(MS_data)

In [3]:
##Extract all data from Randomization.
Rand_data = project.export_records(forms=['randomization'])

#convert data to Dataframe.
Rand_data=pd.DataFrame(Rand_data)

In [4]:

Rand_data=Rand_data[['con_participantid_q1','redcap_event_name','ran_date','rnd_arm']]
Rand=pd.DataFrame(Rand_data)


#st_date as a date data type
Rand_data['ran_date'] = pd.to_datetime(Rand_data['ran_date'], errors='coerce')


rnd_data = Rand_data[
    (Rand['redcap_event_name'] == 'week_20_arm_1') &
    (Rand_data['rnd_arm']!="")
]

rnd_data=pd.DataFrame(rnd_data)

In [5]:
#Randomization group mapping
rnd_group = {
'1':'B',
'2':'N',
'3':'P',
'4':'A',
'5':'X',
'6':'Y',
'7':'C',
'8':'U',
'9':'E',
'10':'K',
'11':'J',
'12':'H',
'13':'W',
'14':'T',
'15':'L'
}
rnd_data['rnd_arm'] = rnd_data['rnd_arm'].map(rnd_group)


In [6]:
rnd_data=pd.DataFrame(rnd_data)

In [7]:

#export infant ID
inf_dob = project.export_records(forms=['infantid', 'delivery'])

# Extract infant delivery and infant data
inf_dob = project.export_records(forms=['infantid', 'delivery'])
inf_del = pd.DataFrame(inf_dob)

# Select relevant columns
inf_data = inf_del[['con_participantid_q1', 'infantid', 'del_ddate_q7']]
df_infant = pd.DataFrame(inf_data)

# Convert delivery date to datetime data type
df_infant['del_ddate_q7'] = pd.to_datetime(df_infant['del_ddate_q7'], errors='coerce')

# Calculate supplementation end date
df_infant['end_date'] = df_infant['del_ddate_q7'] + pd.DateOffset(months=6)

# Assuming rnd_data is already defined and contains the columns 'con_participantid_q1', 'ran_date', and 'rnd_arm'
Rand1=rnd_data[['con_participantid_q1','ran_date','rnd_arm','redcap_event_name']]
# Merge df_infant with rnd_data to filter only the rows with minimum instances
Randomized_part = pd.merge( df_infant,Rand1, on=['con_participantid_q1'] , how='inner')

# Select relevant columns from the merged dataframe
randomised = Randomized_part[['con_participantid_q1', 'infantid', 'ran_date', 'rnd_arm', 'del_ddate_q7', 'end_date']]
randomised=randomised.drop_duplicates()

# Print the columns of the merged dataframe
print(Randomized_part.columns)

randomised_df = pd.DataFrame({
    'Participant ID':randomised['con_participantid_q1'],
    'Infant ID': randomised['infantid'],
    'Randomization date': randomised['ran_date'],
    'Randomization Arm': randomised['rnd_arm'],
    'Infant DoB': randomised['del_ddate_q7'],
    'Supplement end date': randomised['end_date'],
})

# Save the DataFrame to a CSV file
randomised_df.to_csv('Supplementaton report.csv', index=False)


Index(['con_participantid_q1', 'infantid', 'del_ddate_q7', 'end_date',
       'ran_date', 'rnd_arm', 'redcap_event_name'],
      dtype='object')


In [8]:
import pandas as pd

# Specify the fields you want to extract based on logic
MS_1_dataT = MatSup_data[['con_participantid_q1', 'redcap_event_name', 'redcap_repeat_instrument', 'redcap_repeat_instance', 'ms_dvisit_q1','ms_datetaken', 'ms_sup_administered_q2']]
MS_1_dataT = pd.DataFrame(MS_1_dataT)

# Convert 'ms_datetaken' and 'ms_dvisit_q1' to date data type
MS_1_dataT['ms_datetaken'] = pd.to_datetime(MS_1_dataT['ms_datetaken'], errors='coerce')
MS_1_dataT['ms_dvisit_q1'] = pd.to_datetime(MS_1_dataT['ms_dvisit_q1'], errors='coerce')

# Filter out rows where 'redcap_repeat_instance' is an empty string
MS_1_dataT = MS_1_dataT[MS_1_dataT['redcap_repeat_instance'] != '']

# Convert 'redcap_repeat_instance' to integer type
MS_1_dataT['redcap_repeat_instance'] = MS_1_dataT['redcap_repeat_instance'].astype(int)

# Find the minimum instance number for each ID
min_instancesT = MS_1_dataT.groupby('con_participantid_q1')['redcap_repeat_instance'].min().reset_index()

# Merge min_instancesT with MS_1_dataT to filter only the rows with minimum instances
MS_first_instanceT = pd.merge(MS_1_dataT, min_instancesT, on=['con_participantid_q1', 'redcap_repeat_instance'], how='inner')

MS_first_instanceT = pd.DataFrame(MS_first_instanceT)

In [9]:
# Specify the fields you want to extract base on logics.
MS_data=MatSup_data[['con_participantid_q1','redcap_event_name','redcap_repeat_instrument','ms_sup_administered_q2','ms_reason_not_admin_q2a','ms_takenafterfast','ms_datetaken','ms_completed_by']]
MS_data=pd.DataFrame(MS_data)

#st_date as a date data type
MS_data['hrf_date'] = pd.to_datetime(MS_data['ms_datetaken'], errors='coerce')


MS_taken = MS_data[
    (MS_data['ms_sup_administered_q2']=="1") |
    (MS_data['ms_takenafterfast']=="1")
]

MS_taken=pd.DataFrame(MS_taken)

In [10]:
##SUM OF ALL IP CONSUMED BY EACH PARTICIPANT
MS_taken=pd.DataFrame(MS_taken)

# Group by 'participantID' and count the occurrences
sum_ms_taken = MS_taken.groupby('con_participantid_q1').size().reset_index(name='total_ms_taken')

print(sum_ms_taken)


    con_participantid_q1  total_ms_taken
0              IN-M-001E             317
1              IN-M-002H             292
2              IN-M-004F             306
3              IN-M-005J             307
4              IN-M-007A             258
..                   ...             ...
241            IN-M-415L              10
242            IN-M-416K              30
243            IN-M-418C               9
244            IN-M-441F               7
245            IN-M-452K               2

[246 rows x 2 columns]


In [11]:
MS_not_taken = MS_data[
    ((MS_data['ms_sup_administered_q2']=="0") & (MS_data['ms_reason_not_admin_q2a']!="6")) |
    ((MS_data['ms_reason_not_admin_q2a']=="6") & (MS_data['ms_takenafterfast']=="0"))
]

In [12]:
#SUM OF ALL THE MISSED IP'S FOR EACH PARTICIPANT
ms_not_taken=pd.DataFrame(MS_not_taken)

# Group by 'participantID' and count the occurrences
sum_ms_not_taken = ms_not_taken.groupby('con_participantid_q1').size().reset_index(name='total_ms_not_taken')

In [13]:
##MERGE ALL NECESSARY DATAFRAME FOR THE FINAL REPORT
#merge infant_data to delivery_data base on participant ID
rand_supp = rnd_data.merge (sum_ms_taken, on='con_participantid_q1', how='left')

#Merge the two DataFrames on 'con_participant_id' column
ms_rnd_merged = rand_supp.merge(sum_ms_not_taken, on='con_participantid_q1', how='left')

#Merge the two DataFrames on 'con_participant_id' column
ms_rnd_merged = ms_rnd_merged.merge(MS_first_instanceT, on='con_participantid_q1', how='left')

# Select the desired columns
ms_rnd_log = ms_rnd_merged[['con_participantid_q1','rnd_arm','ms_dvisit_q1','total_ms_taken', 'total_ms_not_taken']]

In [14]:
IP_log = pd.DataFrame({
    'Participant ID':ms_rnd_log['con_participantid_q1'],
    'Randomization Arm': ms_rnd_log['rnd_arm'],
    'First visit date': ms_rnd_log['ms_dvisit_q1'],
    'IP Consumed': ms_rnd_log['total_ms_taken'],
    'IP Not Consumed': ms_rnd_log['total_ms_not_taken'],
})

# Save the DataFrame to a CSV file
IP_log.to_csv('IP Log.csv', index=False)